<a href="https://colab.research.google.com/github/sejal2005/Android-BMI-Calculator/blob/main/Legal_Assistant_Proof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === UPLOAD YOUR TEST FILE ===
from google.colab import files

print("UPLOAD Employment_Agreement_1.txt or any .txt file")
uploaded = files.upload()

# Save the file name
test_file = list(uploaded.keys())[0]
print(f"\nUploaded: {test_file}")

# Read the text
with open(test_file, 'r', encoding='utf-8') as f:
    original_text = f.read()

print(f"First 200 characters:\n{original_text[:200]}...")

UPLOAD Employment_Agreement_1.txt or any .txt file


Saving Employment Agreement_1.txt to Employment Agreement_1 (2).txt

Uploaded: Employment Agreement_1 (2).txt
First 200 characters:
1 
 EMPLOYMENT AGREEMENT  
 
This agreement lays down the terms of employment, agreed upon by the employer 
and employee.  Whether stated explicitly in the agreement or not , both the 
employee  and t...


In [ ]:
# === INSTALL REQUIRED PACKAGES ===
!pip install -q transformers torch evaluate py-readability-metrics rouge-score nltk spacy
!python -m spacy download en_core_web_sm -q
import nltk
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

print("All packages installed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
All packages installed!


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("This is a test sentence.")
print("spacy WORKS! Tokens:", [token.text for token in doc])

spacy WORKS! Tokens: ['This', 'is', 'a', 'test', 'sentence', '.']


In [ ]:
from transformers import pipeline
import time
import evaluate
from readability import Readability
import os
import tempfile

rouge = evaluate.load('rouge')
print("Setup ready!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Setup ready!


In [ ]:
# === FIX NLTK ERROR ===
import nltk
nltk.download('punkt_tab')
print("NLTK fixed!")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


NLTK fixed!


In [ ]:
# === FINAL COMPARISON: 3 MODELS ON FULL CONTRACT ===
!pip install -q transformers torch evaluate py-readability-metrics rouge-score

from transformers import pipeline
import time
import evaluate
from readability import Readability
import os

rouge = evaluate.load('rouge')

# === FULL TEXT (NO TRUNCATION) ===
full_text = original_text  # Your 1,250+ word file
word_count = len(full_text.split())
print(f"FILE: {test_file}")
print(f"Word count: {word_count}\n")

# === CHUNK FUNCTION ===
def chunk_text(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(full_text)

results = []

# === 1. PEGASUS ===
print("1. RUNNING PEGASUS...")
try:
    pegasus = pipeline("summarization", model="google/pegasus-xsum", device=-1)
    pegasus_summaries = []
    start = time.time()
    for chunk in chunks:
        if len(chunk.split()) < 100:  # Pegasus fails here
            raise ValueError("100 words required")
        pegasus_summaries.append(pegasus(chunk, max_length=100, min_length=30)[0]['summary_text'])
    pegasus_summary = " ".join(pegasus_summaries)
    pegasus_time = time.time() - start
    pegasus_rouge = rouge.compute(predictions=[pegasus_summary], references=[full_text])['rouge1']
    pegasus_read = Readability(pegasus_summary).flesch_kincaid().score
    pegasus_status = "SUCCESS"
except Exception as e:
    pegasus_summary = f"FAILED: {str(e)[:50]}"
    pegasus_time = 999
    pegasus_rouge = 0
    pegasus_read = 99
    pegasus_status = "FAILED"
results.append(("Pegasus", pegasus_time, pegasus_rouge, pegasus_read, pegasus_status))

# === 2. T5 ===
print("\n2. RUNNING T5...")
try:
    t5 = pipeline("summarization", model="t5-small", device=-1)
    t5_summaries = []
    start = time.time()
    for chunk in chunks:
        t5_summaries.append(t5("summarize: " + chunk, max_length=100, min_length=30)[0]['summary_text'])
    t5_summary = " ".join(t5_summaries)
    t5_time = time.time() - start
    t5_rouge = rouge.compute(predictions=[t5_summary], references=[full_text])['rouge1']
    t5_read = Readability(t5_summary).flesch_kincaid().score
    t5_status = "SUCCESS"
except Exception as e:
    t5_summary = f"FAILED: {str(e)[:50]}"
    t5_time = 999
    t5_rouge = 0
    t5_read = 99
    t5_status = "FAILED"
results.append(("T5", t5_time, t5_rouge, t5_read, t5_status))

# === 3. DISTILBART ===
print("\n3. RUNNING DISTILBART...")
distilbart = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)
db_summaries = []
start = time.time()
for chunk in chunks:
    if len(chunk.split()) < 30:
        continue
    db_summaries.append(distilbart(chunk, max_length=100, min_length=30)[0]['summary_text'])
db_summary = " ".join(db_summaries)
db_time = time.time() - start
db_rouge = rouge.compute(predictions=[db_summary], references=[full_text])['rouge1']
db_read = Readability(db_summary).flesch_kincaid().score
db_status = "SUCCESS"
results.append(("DistilBART", db_time, db_rouge, db_read, db_status))

# === PRINT TABLE ===
print("\n" + "="*80)
print(" FINAL PROOF: MODEL COMPARISON ON FULL INDIAN LABOUR CONTRACT")
print("="*80)
print(f"{'Model':<12} {'Time (s)':<10} {'ROUGE-1':<10} {'Readability':<12} {'Status'}")
print("-"*80)
for name, t, r, rd, s in results:
    print(f"{name:<12} {t:<10.1f} {r:<10.3f} {rd:<12.1f} {s}")
print("="*80)

# === SAVE TO FILE ===
with open("comparison_proof.txt", "w") as f:
    f.write("PRACTICAL PROOF\n\n")
    f.write(f"File: {test_file} ({word_count} words)\n\n")
    for name, _, _, _, _ in results:
        summary = globals().get(f"{name.lower()}_summary", "")
        f.write(f"{name} SUMMARY:\n{summary[:300]}...\n\n")
    f.write("TABLE:\n")
    for name, t, r, rd, s in results:
        f.write(f"{name}: {t:.1f}s | ROUGE-1: {r:.3f} | Read: {rd:.1f} | {s}\n")

# Convert to PDF
!apt install -q texlive-xetex texlive-fonts-recommended texlive-plain-generic
!pandoc comparison_proof.txt -o PROOF_OF_MODEL_SELECTION.pdf
from google.colab import files
files.download('PROOF_OF_MODEL_SELECTION.pdf')

FILE: Employment Agreement_1 (2).txt
Word count: 2723

1. RUNNING PEGASUS...


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors



2. RUNNING T5...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

Device set to use cpu



 FINAL PROOF: MODEL COMPARISON ON FULL INDIAN LABOUR CONTRACT
Model        Time (s)   ROUGE-1    Readability  Status
--------------------------------------------------------------------------------
Pegasus      999.0      0.000      99.0         FAILED
T5           35.9       0.254      12.2         SUCCESS
DistilBART   101.6      0.256      15.2         SUCCESS
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-in

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === FULL SUMMARIES: PRINT & SAVE (NO TRUNCATION) ===
print("\n" + "="*80)
print(" FULL T5 SUMMARY (UNTRUNCATED)")
print("="*80)
print(t5_summary)  # Full T5 summary

print("\n" + "="*80)
print(" FULL DISTILBART SUMMARY (UNTRUNCATED)")
print("="*80)
print(db_summary)  # Full DistilBART summary

# === SAVE FULL SUMMARIES TO FILE ===
with open("FULL_SUMMARIES.txt", "w") as f:
    f.write("FULL SUMMARIES\n\n")
    f.write(f"FILE: {test_file} ({word_count} words)\n\n")
    f.write("T5 SUMMARY (FULL):\n")
    f.write(t5_summary + "\n\n")
    f.write("DISTILBART SUMMARY (FULL):\n")
    f.write(db_summary + "\n\n")
    f.write("METRICS:\n")
    f.write(f"T5: {t5_time:.1f}s | ROUGE-1: {t5_rouge:.3f} | Read: {t5_read:.1f}\n")
    f.write(f"DistilBART: {db_time:.1f}s | ROUGE-1: {db_rouge:.3f} | Read: {db_read:.1f}\n")

# Convert to PDF
!apt install -q texlive-xetex texlive-fonts-recommended texlive-plain-generic
!pandoc FULL_SUMMARIES.txt -o FULL_SUMMARIES.pdf
from google.colab import files
files.download('FULL_SUMMARIES.pdf')


 FULL T5 SUMMARY (UNTRUNCATED)
the agreement is entered into on this ___ day of ________, 20__, BY AND BETWEEN _____________ . the parties hereto desire to enter into this agreement to define and set forth the terms and conditions of the employment of the Employee by the Company . a. Upon execution of this Agreement, the employee would be posted as the _______ of the Company . the Company may change the employee's above mentioned post or location based on the Company's production, operation or working requirements . it is understood and agreed that the first __________ days of employment shall constitute a probationary period (“ Probationary Period”) . the Employee shall not engage in any kind of theft , fraud , misrepresentation or any other illegal act . he/she will not be liable for such an act done at his own risk . the employee promises to never engage in theft of the Employer’s property . the employee must submit to the Employer a self -certification form . if such absence lasts

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === FINAL FAIR COMPARISON: LEGAL-PEGASUS vs T5 vs DISTILBART ===
!pip install -q transformers torch evaluate py-readability-metrics rouge-score sentencepiece

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import evaluate
from readability import Readability
import time
import glob
import os

rouge = evaluate.load('rouge')

# === LOAD YOUR DOCUMENT ===
text_files = glob.glob("/content/*.txt")
documents = []
for file in text_files:
    with open(file, "r", encoding="utf-8") as f:
        documents.append(f.read())

if not documents:
    raise FileNotFoundError("No .txt file found in /content/")
original_text = documents[0]
print(f"Loaded: {text_files[0]} | Words: {len(original_text.split())}\n")

# === CHUNKING FUNCTION (512 tokens → ~400 words) ===
def nest_sentences(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = nest_sentences(original_text)
print(f"Chunks created: {len(chunks)}\n")

# === 1. LEGAL-PEGASUS (YOUR WORKING CODE) ===
print("1. RUNNING LEGAL-PEGASUS (nsi319/legal-pegasus)...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
    model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus").to(device)

    def legal_summarize(text):
        inputs = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True).to(device)
        summary_ids = model.generate(
            inputs,
            num_beams=9,
            length_penalty=0.1,
            min_length=50,
            max_length=150,
            early_stopping=True
        )
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

    lp_summaries = []
    start = time.time()
    for i, chunk in enumerate(chunks):
        if len(chunk.split()) < 100:
            print(f"  Skipping chunk {i+1} (<100 words)")
            continue
        lp_summaries.append(legal_summarize(chunk))
    legal_summary = " ".join(lp_summaries)
    lp_time = time.time() - start
    lp_rouge = rouge.compute(predictions=[legal_summary], references=[original_text])['rouge1']
    lp_read = Readability(legal_summary).flesch_kincaid().score if len(legal_summary) > 50 else 99
    lp_status = "SUCCESS"
except Exception as e:
    legal_summary = f"FAILED: {str(e)[:100]}"
    lp_time, lp_rouge, lp_read = 999, 0, 99
    lp_status = "FAILED"

# === 2. T5 ===
print("\n2. RUNNING T5...")
t5_pipe = pipeline("summarization", model="t5-small", device=0 if device == "cuda" else -1)
t5_summaries = []
start = time.time()
for chunk in chunks:
    if len(chunk.split()) < 30:
        continue
    t5_summaries.append(t5_pipe("summarize: " + chunk, max_length=100, min_length=30)[0]['summary_text'])
t5_summary = " ".join(t5_summaries)
t5_time = time.time() - start
t5_rouge = rouge.compute(predictions=[t5_summary], references=[original_text])['rouge1']
t5_read = Readability(t5_summary).flesch_kincaid().score if len(t5_summary) > 50 else 99

# === 3. DISTILBART ===
print("\n3. RUNNING DISTILBART...")
db_pipe = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0 if device == "cuda" else -1)
db_summaries = []
start = time.time()
for chunk in chunks:
    if len(chunk.split()) < 30:
        continue
    db_summaries.append(db_pipe(chunk, max_length=100, min_length=30)[0]['summary_text'])
db_summary = " ".join(db_summaries)
db_time = time.time() - start
db_rouge = rouge.compute(predictions=[db_summary], references=[original_text])['rouge1']
db_read = Readability(db_summary).flesch_kincaid().score if len(db_summary) > 50 else 99

# === PRINT FULL SUMMARIES ===
print("\n" + "="*100)
print(" FULL SUMMARIES")
print("="*100)
print("LEGAL-PEGASUS:\n", legal_summary[:1000], "..." if len(legal_summary) > 1000 else "")
print("\nT5:\n", t5_summary[:1000], "..." if len(t5_summary) > 1000 else "")
print("\nDISTILBART:\n", db_summary[:1000], "..." if len(db_summary) > 1000 else "")

# === FINAL TABLE ===
print("\n" + "="*100)
print(" FINAL FAIR COMPARISON")
print("="*100)
print(f"{'Model':<15} {'Time (s)':<10} {'ROUGE-1':<10} {'Readability':<12} {'Status'}")
print("-"*100)
print(f"{'Legal-Pegasus':<15} {lp_time:<10.1f} {lp_rouge:<10.3f} {lp_read:<12.1f} {lp_status}")
print(f"{'T5':<15} {t5_time:<10.1f} {t5_rouge:<10.3f} {t5_read:<12.1f} SUCCESS")
print(f"{'DistilBART':<15} {db_time:<10.1f} {db_rouge:<10.3f} {db_read:<12.1f} SUCCESS")
print("="*100)

# === SAVE TO PDF ===
with open("FINAL_LEGAL_COMPARISON.txt", "w") as f:
    f.write("FINAL LEGAL MODEL COMPARISON\n\n")
    f.write(f"File: {text_files[0]} | Words: {len(original_text.split())}\n\n")
    f.write("LEGAL-PEGASUS SUMMARY:\n" + legal_summary + "\n\n")
    f.write("T5 SUMMARY:\n" + t5_summary + "\n\n")
    f.write("DISTILBART SUMMARY:\n" + db_summary + "\n\n")
    f.write("METRICS:\n")
    f.write(f"Legal-Pegasus: {lp_time:.1f}s | ROUGE-1: {lp_rouge:.3f} | Read: {lp_read:.1f} | {lp_status}\n")
    f.write(f"T5: {t5_time:.1f}s | ROUGE-1: {t5_rouge:.3f} | Read: {t5_read:.1f}\n")
    f.write(f"DistilBART: {db_time:.1f}s | ROUGE-1: {db_rouge:.3f} | Read: {db_read:.1f}\n")

!apt install -q texlive-xetex texlive-fonts-recommended texlive-plain-generic
!pandoc FINAL_LEGAL_COMPARISON.txt -o FINAL_LEGAL_COMPARISON.pdf
from google.colab import files
files.download('FINAL_LEGAL_COMPARISON.pdf')

Loaded: /content/Employment Agreement_1.txt | Words: 2723

Chunks created: 7

1. RUNNING LEGAL-PEGASUS (nsi319/legal-pegasus)...
Using device: cpu

2. RUNNING T5...


Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (628 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(


3. RUNNING DISTILBART...


Device set to use cpu



 FULL SUMMARIES
LEGAL-PEGASUS:
 A private limited company incorporated under the Companies Act, 1956 has entered into an employment agreement with a former employee of the company. The agreement provides for the employment of the employee by the company in accordance with the terms and conditions of the employment of the employee. The terms and conditions of the employment of the employee are set out below. The parties have the duty of mutual confidence and trust, and to make only lawful and reasonable demands on each other. The following terms and conditions are set out under the provisions of the Employment Act of 2006 and the terms and conditions of this Agreement. The terms and conditions of this Agreement are set out below. The terms and conditions of this Agreement are subject to the approval of the Board of Directors of the Company. The following rules and policies govern the conduct of employees employed by the company. The following rules and policies govern the conduct of em

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>